In [22]:
import pandas as pd
pop = pd.read_csv('../data/mock_sysselsatte/population_2019_30000.csv')
comp = pd.read_csv('../data/mock_sysselsatte/companies_2019_30000.csv')

### Nedfiltrering, en essensiell kompetanse

In [23]:
# De fra 20 tom. 30 år i datasettet
pop_20_30 = pop[    (pop['age'] >= 20)  &  (pop['age'] <= 30)    ]
# Se på aldrene som nå eksisterer i datasettet, sortert
sorted(  pd.unique( pop_20_30['age'] )  )

[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

### Lag grupperingsvariabler etter en inndeling av alder

In [26]:
pop2 = pop.copy()
# Litt juks, ta bort siste tegnet i alderen
pop2['age_group'] = pop2['age'].astype(str).str[:-1]
# De som er tomme på alder nå, får gruppen "0"
pop2['age_group'] = pop2['age_group'].replace('', '0')
# Concatenate en 0 på slutten
pop2['age_group'] = pop2['age_group'] + '0' 
# Se på alder og aldersgruppe
pop2[['age', 'age_group']].sample(10)

,age,age_group
6741,21,20
10686,51,50
11189,62,60
14786,82,80
28786,79,70
11631,48,40
21296,35,30
18682,22,20
23900,48,40
29289,80,80


### Litt mer sofistikert eller "generell"

In [53]:
pop3 = pop.copy()

In [54]:
pop3.dtypes

Unnamed: 0        int64
id                int64
work_id         float64
sex              object
age               int64
year_birth        int64
work_percent    float64
dtype: object

In [55]:
last = ''
# Alle tall fra 0 til, men ikke med 120, med ett "step" på 10
for ar in range(0, 120, 10):
    print(ar)
    # Hopp over første loopen, men gå inn her om forrige var 0
    if last or last == 0:
        # Om forrige var 0 prøver vi å få med de som var 0 år
        if last == 0:
            pop3.loc[ ( pop3['age'] >= last) & ( pop3['age'] <= ar ), 'age_group'] = f'{last} - {ar}' 
            print(last, ar)
            print(f"Forrige år var {last}, og vi har da spesielle regler for å få med de som er 0 år")
        # Dette er da for de gruppene, hvor første år ikke er 0
        else:
            # Legg merke til at likhetstegnet er borte fra den første parantesen
            pop3.loc[ ( pop3['age'] > last) & ( pop3['age'] <= ar ), 'age_group'] = f'{last + 1} - {ar}' 
            print(last + 1, ar)
            print(f"Forrige år var ikke 0, da starter vi 1 over {last}")
    last = ar

0
10
0 10
Forrige år var 0, og vi har da spesielle regler for å få med de som er 0 år
20
11 20
Forrige år var ikke 0, da starter vi 1 over 10
30
21 30
Forrige år var ikke 0, da starter vi 1 over 20
40
31 40
Forrige år var ikke 0, da starter vi 1 over 30
50
41 50
Forrige år var ikke 0, da starter vi 1 over 40
60
51 60
Forrige år var ikke 0, da starter vi 1 over 50
70
61 70
Forrige år var ikke 0, da starter vi 1 over 60
80
71 80
Forrige år var ikke 0, da starter vi 1 over 70
90
81 90
Forrige år var ikke 0, da starter vi 1 over 80
100
91 100
Forrige år var ikke 0, da starter vi 1 over 90
110
101 110
Forrige år var ikke 0, da starter vi 1 over 100


In [56]:
pop3[['age', 'age_group']].sample(10)

,age,age_group
29807,81,81 - 90
17264,6,0 - 10
8560,35,31 - 40
21845,30,21 - 30
3639,28,21 - 30
3859,35,31 - 40
24167,54,51 - 60
26163,62,61 - 70
10745,55,51 - 60
24937,60,51 - 60


### Sett en verdi i en kolonne basert på en verdi i en annen kolonne, med ett .loc-søk

In [57]:
pop3.loc[pop3['age'] == 0, 'status'] = "nyfodt"

### Det samme, bare at vi looper over en dict med "nivåer"

In [61]:
statuser = {
    13 : 'tenaring',
    18 : 'voksen',
    30 : 'middelaldrende',
    60 : 'eldre'
}

In [62]:
for k, i in statuser.items():
    print(k, i)
    pop3.loc[pop3['age'] >= k, 'status'] = i

13 tenaring
18 voksen
30 middelaldrende
60 eldre


In [63]:
pop3[['age','status']].sample(10)

,age,status
3075,16,tenaring
22906,44,middelaldrende
28841,75,eldre
14323,77,eldre
28799,78,eldre
20092,25,voksen
15090,99,eldre
22394,35,middelaldrende
6390,35,middelaldrende
10213,47,middelaldrende


In [64]:
# Kan vi stole på rekkefølgen i en dict? Eller er det en annen datatype vi heller bør satse på da?
from collections import OrderedDict
help(OrderedDict)

Help on class OrderedDict in module collections:

class OrderedDict(builtins.dict)
 |  Dictionary that remembers insertion order
 |  
 |  Method resolution order:
 |      OrderedDict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __reduce__(...)
 |      Return state information for pickling
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __reversed__(..